In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uproot

import os
import sys
sys.path.append(os.path.abspath(".."))

from helpers.plotting_3d import plot_event


In [2]:
f = uproot.open("../input_files/bdt_convert_superunified_bnb_ncpi0_full_spacepoints.root")

T_bdt_vars = [              # variables involved with BDT training (if you want to train your own BDT, lots of these variables will be useful)
                            # many of these variables describe specific features of the WC spacepoints using this code: https://github.com/BNLIF/wire-cell-pid/blob/master/src/NeutrinoID_nue_tagger.h
                            # here, we just include higher level outputs:
    "nue_score",                    # BDT score for nue selection, used for the WC inclusive nueCC analysis
    "numu_score",                   # BDT score for numu selection, used for the WC inclusive numuCC selections
    "nc_delta_score",               # BDT score for NC Delta selection
    "nc_pio_score",                 # BDT score for NC pi0 selection
    "numu_cc_flag",                 # flag, -1 means not generic selected, 0 means generic selected, 1 means cut-based numuCC selected. We often use "numu_cc_flag >= 0" to apply generic neutrino selection.
]

T_eval_vars = [             # variables involved with low level reconstruction and truth information
    "run",                          # run number
    "subrun",                       # subrun number
    "event",                        # event number
    "match_isFC",                   # reconstructed cluster is fully contained (FC), boolean
    "truth_nuEnergy",               # true neutrino energy (MeV)
    "truth_nuPdg",                  # true neutrino pdg code
    "truth_isCC",                   # true interaction type is charged current, boolean
    "match_completeness_energy",    # the true energy deposited in the clusters that are 3D-matched with the reconstructed neutrino clusters (MeV)
    "truth_energyInside",           # the true energy deposited in the TPC Fiducial Volume (MeV)
    "truth_vtxInside",              # boolean, true neutrino vertex is inside the TPC Fiducial Volume
    "weight_cv",                    # GENIE MicroBooNE tune event weight (which should be corrected by also using weight_spline)
    "weight_spline",                # additional weight to correct the GENIE tune for certain events
]
T_eval_data_vars = [        # same as above, but for data files we do not attempt to load any truth information
    "run",
    "subrun",
    "event",
    "match_isFC",
]

T_kine_vars = [             # variables involved with kinematic reconstruction
    "kine_reco_Enu",                # reconstructed neutrino energy (MeV). "kine_reco_Enu > 0" is another way to apply generic neutrino selection.
    "kine_energy_particle",         # energy of each reco particle
    "kine_particle_type",           # pdg code of each reco particle
]

T_pf_vars = [               # variables involved with individual particles
    "truth_NprimPio",
    "truth_NCDelta",
    "reco_nuvtxX",
    "reco_nuvtxY",
    "reco_nuvtxZ",
    "reco_muonMomentum",            # reconstructed muon momentum 4-vector (p_x, p_y, p_z, p_t), in (GeV/c, GeV/c, GeV/c, GeV)
    "reco_showerMomentum",          # reconstructed primary shower momentum 4-vector (p_x, p_y, p_z, p_t), in (GeV/c, GeV/c, GeV/c, GeV)
    "reco_showervtxX",
    "reco_showervtxY",
    "reco_showervtxZ",
    "truth_vtxX",                   # true neutrino vertex x (cm)
    "truth_vtxY",                   # true neutrino vertex y (cm)
    "truth_vtxZ",                   # true neutrino vertex z (cm)
    "truth_corr_nuvtxX",            # true neutrino vertex x (cm), corrected for SCE
    "truth_corr_nuvtxY",            # true neutrino vertex y (cm), corrected for SCE
    "truth_corr_nuvtxZ",            # true neutrino vertex z (cm), corrected for SCE

    # These variables are related to individual true particles
    "truth_Ntrack",
    "truth_id",
    "truth_pdg",
    "truth_mother",
    "truth_startMomentum",
    "truth_startXYZT",
    "truth_endXYZT",

    # These variables are related to individual reco particles
    "reco_Ntrack",
    "reco_id",
    "reco_pdg",
    "reco_mother",
    "reco_startMomentum",
    "reco_startXYZT",
    "reco_endXYZT",
]

T_pf_data_vars = [          # same as above, but for data files we do not attempt to load any truth information
    "reco_nuvtxX",
    "reco_nuvtxY",
    "reco_nuvtxZ",
    "reco_muonMomentum",
    "reco_showerMomentum",
    "reco_showervtxX",
    "reco_showervtxY",
    "reco_showervtxZ",
    "reco_Ntrack",
    "reco_id",
    "reco_pdg",
    "reco_mother",
    "reco_startMomentum",
    "reco_startXYZT",
    "reco_endXYZT",
]

T_spacepoints_vars = [
    "Trec_spacepoints_x",
    "Trec_spacepoints_y",
    "Trec_spacepoints_z",
    "Trec_spacepoints_q",
    "Treccharge_spacepoints_x",
    "Treccharge_spacepoints_y",
    "Treccharge_spacepoints_z",
    "Treccharge_spacepoints_q",
    "Trecchargeblob_spacepoints_x",
    "Trecchargeblob_spacepoints_y",
    "Trecchargeblob_spacepoints_z",
    "Trecchargeblob_spacepoints_q",
    "Tcluster_spacepoints_x",
    "Tcluster_spacepoints_y",
    "Tcluster_spacepoints_z",
    "Tcluster_spacepoints_q",
    "TrueEDep_spacepoints_startx",
    "TrueEDep_spacepoints_starty",
    "TrueEDep_spacepoints_startz",
]

vars = {}
vars.update(f["wcpselection"]["T_BDTvars"].arrays(T_bdt_vars, library="np", entry_start=0, entry_stop=100))
vars.update(f["wcpselection"]["T_eval"].arrays(T_eval_vars, library="np", entry_start=0, entry_stop=100))
vars.update(f["wcpselection"]["T_KINEvars"].arrays(T_kine_vars, library="np", entry_start=0, entry_stop=100))
vars.update(f["wcpselection"]["T_PFeval"].arrays(T_pf_vars, library="np", entry_start=0, entry_stop=100))
vars.update(f["wcpselection"]["T_spacepoints"].arrays(T_spacepoints_vars, library="np", entry_start=0, entry_stop=100))
file_POT = np.sum(f["wcpselection"]["T_pot"].arrays("pot_tor875good", library="np")["pot_tor875good"])
for col in vars:
    vars[col] = vars[col].tolist()
wc_df = pd.DataFrame(vars)

all_df = wc_df

In [3]:
# event with a proton blip upstream of one shower
event_index = 15

# each entry is name : (points, color, cmap, size, visible)
points_dic = {}

points_dic["reco_nu_vtx"] = (
    np.array([all_df["reco_nuvtxX"].to_numpy()[event_index], all_df["reco_nuvtxY"].to_numpy()[event_index], all_df["reco_nuvtxZ"].to_numpy()[event_index]]),
    "green",
    None,
    10,
    "legendonly"
)
points_dic["true_nu_vtx"] = (
    np.array([all_df["truth_vtxX"].to_numpy()[event_index], all_df["truth_vtxY"].to_numpy()[event_index], all_df["truth_vtxZ"].to_numpy()[event_index]]),
    "red",
    None,
    10,
    "legendonly"
)
points_dic["true_corr_nu_vtx"] = (
    np.array([all_df["truth_corr_nuvtxX"].to_numpy()[event_index], all_df["truth_corr_nuvtxY"].to_numpy()[event_index], all_df["truth_corr_nuvtxZ"].to_numpy()[event_index]]),
    "yellow",
    None,
    10,
    "legendonly"
)
points_dic["Trec_spacepoints"] = (
    np.vstack((all_df["Trec_spacepoints_x"].to_numpy()[event_index], all_df["Trec_spacepoints_y"].to_numpy()[event_index], all_df["Trec_spacepoints_z"].to_numpy()[event_index])).T,
    "red",
    None,
    1,
    "legendonly"
)
points_dic["Treccharge_spacepoints"] = (
    np.vstack((all_df["Treccharge_spacepoints_x"].to_numpy()[event_index], all_df["Treccharge_spacepoints_y"].to_numpy()[event_index], all_df["Treccharge_spacepoints_z"].to_numpy()[event_index])).T,
    "blue",
    None,
    1,
    "legendonly"
)
points_dic["Trecchargeblob_spacepoints"] = (
    np.vstack((all_df["Trecchargeblob_spacepoints_x"].to_numpy()[event_index], all_df["Trecchargeblob_spacepoints_y"].to_numpy()[event_index], all_df["Trecchargeblob_spacepoints_z"].to_numpy()[event_index])).T,
    all_df["Trecchargeblob_spacepoints_q"].to_numpy()[event_index],
    "jet",
    1,
    True
)
points_dic["Tcluster_spacepoints"] = (
    np.vstack((all_df["Tcluster_spacepoints_x"].to_numpy()[event_index], all_df["Tcluster_spacepoints_y"].to_numpy()[event_index], all_df["Tcluster_spacepoints_z"].to_numpy()[event_index])).T,
    "blue",
    None,
    1,
    True
)
points_dic["TrueEDep_spacepoints"] = (
    np.vstack((all_df["TrueEDep_spacepoints_startx"].to_numpy()[event_index], all_df["TrueEDep_spacepoints_starty"].to_numpy()[event_index], all_df["TrueEDep_spacepoints_startz"].to_numpy()[event_index])).T,
    "orange",
    None,
    1,
    True
)

plot_event(event_index, points_dic)